In [1]:
# Importing libraries
import requests
import json
import pandas as pd
import csv
import psycopg2


In [2]:
url = "https://linkedin-data-api.p.rapidapi.com/search-jobs"

querystring = {"keywords":"dataengineer","locationId":"92000000","datePosted":"anyTime","sort":"mostRelevant"}

headers = {
	"x-rapidapi-key": "2be5d5701bmsh9cfd68b52506516p147db3jsn4d8e5fc63a17",
	"x-rapidapi-host": "linkedin-data-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

# Save data into a json file
data = response.json()

# Save data into a file
filename = 'linkedin_jobs.json'
with open(filename, 'w') as file:
    json.dump(data, file)

In [3]:
# Read data into a dataframe
df = pd.read_json('linkedin_jobs.json')

# Print the first 5 rows of the dataframe
df.head()

,success,message,data,total
0,True,,"{'id': '3965578157', 'title': 'Adobe Experienc...",26789
1,True,,"{'id': '268062603', 'title': 'Senior Data Engi...",26789
2,True,,"{'id': '3939160908', 'title': 'Data Engineer',...",26789
3,True,,"{'id': '3961702193', 'title': 'Data Engineer',...",26789
4,True,,"{'id': '3921325243', 'title': 'Snowflake Data ...",26789


In [4]:
# Normalize the data
df = pd.json_normalize(df['data'])

# Print the first 5 rows of the normalized dataframe
df.head()

,id,title,url,referenceId,posterId,location,type,postDate,company.name,company.logo,company.url
0,3965578157,Adobe Experience Platform Data Engineer,https://www.linkedin.com/jobs/view/3965578157,SkIz81QsNR7EP13mo/KDzw==,266779521,"Riga, Riga, Latvia (Hybrid)",Full-time,1 week ago,Swisscom DevOps Center Riga,https://media.licdn.com/dms/image/D4D0BAQEnuAt...,https://www.linkedin.com/company/swisscom-devo...
1,268062603,Senior Data Engineer,https://www.linkedin.com/jobs/view/268062603,SkIz81QsNR7EP13mo/KDzw==,8949551,"Bucharest, Romania (Hybrid)",Full-time,3 weeks ago,TRON,https://media.licdn.com/dms/image/C4D0BAQF9m6U...,https://www.linkedin.com/company/tron-it-consu...
2,3939160908,Data Engineer,https://www.linkedin.com/jobs/view/3939160908,SkIz81QsNR7EP13mo/KDzw==,565752904,"Limassol, Cyprus (Hybrid)",Full-time,1 month ago,Electi Consulting,https://media.licdn.com/dms/image/C4E0BAQHM3Lt...,https://www.linkedin.com/company/electiconsult...
3,3961702193,Data Engineer,https://www.linkedin.com/jobs/view/3961702193,SkIz81QsNR7EP13mo/KDzw==,127448466,"Gothenburg, Västra Götaland County, Sweden (On...",Full-time,2 weeks ago,Cetasol - Maritime Sustainability,https://media.licdn.com/dms/image/C4E0BAQHvFb0...,https://www.linkedin.com/company/cetasol/life
4,3921325243,Snowflake Data Engineer,https://www.linkedin.com/jobs/view/3921325243,SkIz81QsNR7EP13mo/KDzw==,649109939,"Budapest, Hungary (Hybrid)",Full-time,2 months ago,Hiflylabs,https://media.licdn.com/dms/image/C4D0BAQFbZrj...,https://www.linkedin.com/company/hiflylabs/life


In [5]:
# Dataframe to CSV
file_path = 'data_engineer_jobs.csv'

df.to_csv(file_path, index=False)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            25 non-null     object
 1   title         25 non-null     object
 2   url           25 non-null     object
 3   referenceId   25 non-null     object
 4   posterId      25 non-null     object
 5   location      25 non-null     object
 6   type          25 non-null     object
 7   postDate      25 non-null     object
 8   benefits      25 non-null     object
 9   company.name  25 non-null     object
 10  company.logo  25 non-null     object
 11  company.url   25 non-null     object
dtypes: object(12)
memory usage: 2.5+ KB


In [6]:
# Postgres connection
def get_db_connection():
    connection = psycopg2.connect(
        host='localhost',
        database='postgres',
        user='postgres',
        password='postgres'
    )
    return connection

conn = get_db_connection()

In [19]:
# Creating database table
def create_table():
    conn = get_db_connection()
    cursor = conn.cursor()
    create_table_query = '''CREATE SCHEMA IF NOT EXISTS jobs;
                            DROP TABLE IF EXISTS jobs.data_engineer;

                            CREATE TABLE jobs.data_engineer(
                            id VARCHAR PRIMARY KEY,
                            title VARCHAR,
                            url VARCHAR,
                            referenceId VARCHAR,
                            posterId VARCHAR,
                            location VARCHAR,
                            type VARCHAR,
                            postDate VARCHAR,
                            benefits VARCHAR,
                            companyname VARCHAR,
                            companylogo VARCHAR,
                            companyurl VARCHAR
                            );'''
    
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()

create_table()

In [20]:
# Load csv data to database

def load_data_from_csv_to_table(csv_path, table_name):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            placeholder = ','.join(['%s'] * len(row))
            query = f'INSERT INTO {table_name} VALUES ({placeholder});'
            cursor.execute(query, row)
    conn.commit()
    cursor.close()
    conn.close()

In [21]:
# Load  data to database
data_engineer_csv_path = r'./data/data_engineer_jobs.csv'
load_data_from_csv_to_table(data_engineer_csv_path, 'jobs.data_engineer')

In [ ]:
print('Data Loaded success fully')